In [ ]:
a = []
while(1):
    a.append('1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install deepface
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 60.9 MB 1.1 MB/s 
     |████████████████████████████████| 87 kB 4.9 MB/s 
     |████████████████████████████████| 2.3 MB 44.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=cadd69b0a1fabfa0b0daf88d297d80a57ab12181d7ea061e80c3688f6e662c51
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requi

In [ ]:
from deepface import DeepFace
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Conv2D, Flatten, Activation,Dense
from keras.models import load_model
from deepface.extendedmodels import Age,Gender
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np
import tensorflow
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from sklearn import metrics
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
kf=KFold(10,shuffle=True,random_state=42)
sf=StratifiedKFold(10,shuffle=True,random_state=42)
from statistics import mean,stdev,variance 
from keras.utils.vis_utils import plot_model




Directory  /root /.deepface created
Directory  /root /.deepface/weights created


Importing The Data

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Images = drive.CreateFile({'id': '11-GiJV96NwFsM_mhHRtrgxCdUuyHnX5I'});
Images.GetContentFile('Images.npy');
Height = drive.CreateFile({'id': '1X_qil6xJbg38oHTKtcWhZ829KU1NKyw7'});
Height.GetContentFile('Height.npy');
Weight = drive.CreateFile({'id': '1dDMAU_S-BM3nRgMf3oE-9Vx0j54aXmH9'});
Weight.GetContentFile('Weight.npy');
BMI = drive.CreateFile({'id': '1A4o2H_57s6qHHLGWX77tbXb5hbpJSeBM'});
BMI.GetContentFile('BMI.npy');
Processed_Images = drive.CreateFile({'id': '1-1JEzyCiPughcWpv0PSGrAYoj79azImE'});
Processed_Images.GetContentFile('Processed_Images.npy');
Processed_Height = drive.CreateFile({'id': '1-4fEd-2r96n6YRPv8rO5O8_sZ-11y2_G'});
Processed_Height.GetContentFile('Processed_Height.npy');
Processed_BMI = drive.CreateFile({'id': '1-3BET2eDW03-fmxG9B6HjJAIA0uCCw5R'});
Processed_BMI.GetContentFile('Processed_BMI.npy');
Processed_Weight= drive.CreateFile({'id': '1-4kKockjQAtKb0WAUcLUmSxU5oUFkPsN'});
Processed_Weight.GetContentFile('Processed_Weight.npy');

In [ ]:
model=Age.loadModel()
model_gender=Gender.loadModel()
Images=np.load("Images.npy")
Height=np.load("Height.npy")
BMI=np.load('BMI.npy')
Weight=np.load('Weight.npy')
Processed_Images=np.load("Processed_Images.npy")
Processed_Height=np.load("Processed_Height.npy")
Processed_BMI=np.load("Processed_BMI.npy")
Processed_Weight=np.load('Processed_Weight.npy')

age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [00:45<00:00, 11.8MB/s]


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:56<00:00, 9.57MB/s]


Shaping The input Images

In [ ]:
images=[]
for i in range(len(Images)):
  img=cv2.resize(Images[i],(224,224))
  img=img.reshape(224,224,3)
  images.append(img)
images=np.array(images)
images.shape

(1227, 224, 224, 3)

In [ ]:
processed_images=[]
for i in range(len(Processed_Images)):
  img=cv2.resize(Processed_Images[i],(224,224))
  img=img.reshape(224,224,3)
  processed_images.append(img)
processed_images=np.array(processed_images)
processed_images.shape

(1174, 224, 224, 3)

Initial Trial Model

In [ ]:
sliced_loaded_model = Sequential(model.layers[:-3])
for layer in sliced_loaded_model.layers:
    layer.trainable = False
sliced_loaded_model.add(Flatten()) 
sliced_loaded_model.add(Dense(4096,activation='relu'))
sliced_loaded_model.add(Dense(1024,activation='relu'))
sliced_loaded_model.add(Dense(1,activation='linear'))

In [ ]:
model_1=sliced_loaded_model
model_1.compile(loss='mean_absolute_percentage_error', optimizer='adam',metrics='mean_absolute_percentage_error')
fitted_model_1=model_1.fit(images,BMI,epochs=100)

Model Trial with Augmented Images


In [ ]:
model_2=sliced_loaded_model
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)
batches=dataGenerator.flow(images,BMI,batch_size=20)
model_2.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(
    reduction="auto", name="mean_absolute_percentage_error"), optimizer='adam',metrics='mean_absolute_percentage_error')
fitted_model_2=model_2.fit_generator(batches,epochs=100)

Hyperparameter Model with fine tuning finding the best model

In [ ]:
def model_builder(hp):
  model_=Sequential(model.layers[:-3])
  for layer in model_.layers:
    layer.trainable = False
  model_.add(Flatten())
  for i in range(hp.Int('num_layers', 2, 5)): 
    model_.add(keras.layers.Dense(units=hp.Int('units_' + str(i),min_value=256,max_value=4096,step=32), activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid','softmax'], default='relu')))
  model_.add(keras.layers.Dense(1,activation='linear'))
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  optimizer=hp.Choice('optimizer', values=['adam', 'adagrad', 'SGD', 'RMSprop'])
  if optimizer == 'adam':
    optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate)
  elif optimizer == 'adagrad':
    optimizer=keras.optimizers.Adagrad(learning_rate=hp_learning_rate)
  elif optimizer == 'SGD':
    optimizer=keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif optimizer == 'RMSprop':
    optimizer=keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  model_.compile(
      loss=tf.keras.losses.MeanAbsolutePercentageError(
    reduction="auto", name="mean_absolute_percentage_error"),
      optimizer=optimizer,
      metrics=['mean_absolute_percentage_error']
      )
  return model_

Finding Best Bmi model by tuning

In [ ]:
tuner_bmi = kt.Hyperband(model_builder, objective='mean_absolute_percentage_error', max_epochs=10, factor=3, directory='/content/drive/MyDrive/BMI_Face_Data/Data/Test_models_bmi', project_name='/content/drive/MyDrive/BMI_Face_Data/Data/Best_Model_')
tuner_bmi.search(images,BMI, epochs=50, validation_split=0.2)

In [ ]:
tuner_bmi_pre = kt.Hyperband(model_builder, objective='mean_absolute_percentage_error', max_epochs=10, factor=3, directory='/content/drive/MyDrive/BMI_Face_Data/Data/Test_models_bmi_pre', project_name='/content/drive/MyDrive/BMI_Face_Data/Data/Best_Model_pre')
tuner_bmi_pre.search(processed_images,Processed_BMI, epochs=50, validation_split=0.2)

BMI prediction with data augmentation-normal image

In [ ]:
best_parameters_bmi_aug=tuner_bmi.get_best_hyperparameters(num_trials=1)[0]
model_1 = tuner_bmi.hypermodel.build(best_parameters_bmi_aug)
model_1.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.1,rotation_range=20)
score_bmi_aug=[]
fold=0
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("======"*10)
  x_train=images[train]
  y_train=BMI[train]
  x_test=images[test]
  y_test=BMI[test]
  batches=dataGenerator.flow(x_train,y_train,batch_size=20)
  model_1.fit_generator(batches,epochs=100,callbacks=[callback])
  pred=model_1.predict(x_test)
  score_bmi_aug.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_bmi_aug)
print(f'mean={mean(score_bmi_aug)}')
print(f'standard_deviation={stdev(score_bmi_aug)}')
print(f'Variance={variance(score_bmi_aug)}')
plot_model(model_1, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/BMI_Aug.png', show_shapes=True, show_layer_names=True)
model_1.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/BMI_Augmented.h5")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

BMI prediction with data augmentation-processed image

In [ ]:
best_parameters_bmi_aug_pre=tuner_bmi_pre.get_best_hyperparameters(num_trials=1)[0]
model_2 = tuner_bmi_pre.hypermodel.build(best_parameters_bmi_aug_pre)
model_2.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.1,rotation_range=20)
score_bmi_aug_pre=[]
fold=0
for train,test in kf.split(processed_images):
  fold+=1;
  print(fold)
  print("======"*10)
  x_train=processed_images[train]
  y_train=Processed_BMI[train]
  x_test=processed_images[test]
  y_test=Processed_BMI[test]
  batches=dataGenerator.flow(x_train,y_train,batch_size=20)
  model_2.fit_generator(batches,epochs=100,callbacks=[callback])
  pred=model_2.predict(x_test)
  score_bmi_aug_pre.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_bmi_aug_pre)
print(f'mean={mean(score_bmi_aug_pre)}')
print(f'standard_deviation={stdev(score_bmi_aug_pre)}')
print(f'Variance={variance(score_bmi_aug_pre)}')
plot_model(model_2, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/BMI_Aug_Pre.png', show_shapes=True, show_layer_names=True)
model_2.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/BMI_Augmented_Pre.h5")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

NameError: ignored

In [ ]:
print(score_bmi_aug_pre)
print(f'mean={mean(score_bmi_aug_pre)}')
print(f'standard_deviation={stdev(score_bmi_aug_pre)}')
print(f'Variance={variance(score_bmi_aug_pre)}')
plot_model(model_2, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/BMI_Aug_Pre.png', show_shapes=True, show_layer_names=True)
model_2.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/BMI_Augmented_Pre.h5")

[0.24870099327632303, 0.2052054832089708, 0.1879043405855809, 0.3112179699236232, 0.1388783110386099, 0.21253428711907135, 0.405754387696271, 0.14071741691102538, 0.12928341948650218, 0.15413160147656146]
mean=0.21343282107225392
standard_deviation=0.08816476082642141
Variance=0.007773025051580092


BMI model without data augmentation-normal image


In [ ]:
best_parameters_bmi=tuner_bmi.get_best_hyperparameters(num_trials=1)[0]
model_3 = tuner_bmi.hypermodel.build(best_parameters_bmi)
model_3.summary()
score_bmi=[]
fold=0
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("======"*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=images[train]
  y_train=BMI[train]
  x_test=images[test]
  y_test=BMI[test]
  model_3.fit(x_train,y_train,callbacks=[callback],epochs=100)
  pred=model_3.predict(x_test)
  score_bmi.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_bmi)
print(f'mean={mean(score_bmi)}')
print(f'standard_deviation={stdev(score_bmi)}')
print(f'Variance={variance(score_bmi)}')
plot_model(model_3, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/BMI.png', show_shapes=True, show_layer_names=True)
model_3.save('/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/BMI.h5')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

BMI model without data augmentation-normal image

In [ ]:
best_parameters_bmi_pre=tuner_bmi_pre.get_best_hyperparameters(num_trials=1)[0]
model_4 = tuner_bmi_pre.hypermodel.build(best_parameters_bmi_pre)
model_4.summary()
score_bmi_pre=[]
fold=0
for train,test in kf.split(processed_images):
  fold+=1;
  print(fold)
  print("======"*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=processed_images[train]
  y_train=Processed_BMI[train]
  x_test=processed_images[test]
  y_test=Processed_BMI[test]
  model_4.fit(x_train,y_train,callbacks=[callback],epochs=100)
  pred=model_4.predict(x_test)
  score_bmi_pre.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_bmi_pre)
print(f'mean={mean(score_bmi_pre)}')
print(f'standard_deviation={stdev(score_bmi_pre)}')
print(f'Variance={variance(score_bmi_pre)}')
plot_model(model_4, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/BMI_Pre.png', show_shapes=True, show_layer_names=True)
model_4.save('/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/BMI_Pre.h5')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Height Model with fine tuning finding the best model


In [ ]:
tuner_height = kt.Hyperband(model_builder, objective='mean_absolute_percentage_error', max_epochs=10, factor=3, project_name='/content/drive/MyDrive/BMI_Face_Data/Data/Height_Test_models')
tuner_height.search(images,Height, epochs=50, validation_split=0.2)

In [ ]:
tuner_height_pre = kt.Hyperband(model_builder, objective='mean_absolute_percentage_error', max_epochs=10, factor=3,project_name='/content/drive/MyDrive/BMI_Face_Data/Data/Height_Test_models_pre')
tuner_height_pre.search(processed_images,Processed_Height, epochs=50, validation_split=0.2)

Height prediction with data augmentation-normal image

In [ ]:
best_parameters_height_aug=tuner_height.get_best_hyperparameters(num_trials=1)[0]
model_5 = tuner_height.hypermodel.build(best_parameters_height_aug)
model_5.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.1,rotation_range=20)
score_height_aug=[]
fold=0
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("======"*10)
  x_train=images[train]
  y_train=Height[train]
  x_test=images[test]
  y_test=Height[test]
  batches=dataGenerator.flow(x_train,y_train,batch_size=20)
  model_5.fit_generator(batches,epochs=100,callbacks=[callback])
  pred=model_5.predict(x_test)
  score_height_aug.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_height_aug)
print(f'mean={mean(score_height_aug)}')
print(f'standard_deviation={stdev(score_height_aug)}')
print(f'Variance={variance(score_height_aug)}')
plot_model(model_5, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Height_Aug.png', show_shapes=True, show_layer_names=True)
model_5.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Height_Augmented.h5")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Height prediction with data augmentation -processed image

In [ ]:
best_parameters_height_aug_pre=tuner_height_pre.get_best_hyperparameters(num_trials=1)[0]
model_6 = tuner_height_pre.hypermodel.build(best_parameters_height_aug_pre)
model_6.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.1,rotation_range=20)
score_height_aug_pre=[]
fold=0
for train,test in kf.split(processed_images):
  fold+=1;
  print(fold)
  print("======"*10)
  x_train=processed_images[train]
  y_train=Processed_Height[train]
  x_test=processed_images[test]
  y_test=Processed_Height[test]
  batches=dataGenerator.flow(x_train,y_train,batch_size=20)
  model_6.fit_generator(batches,epochs=100,callbacks=[callback])
  pred=model_6.predict(x_test)
  score_height_aug_pre.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_height_aug_pre)
print(f'mean={mean(score_height_aug_pre)}')
print(f'standard_deviation={stdev(score_height_aug_pre)}')
print(f'Variance={variance(score_height_aug_pre)}')
plot_model(model_6, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Height_Aug_Pre.png', show_shapes=True, show_layer_names=True)
model_6.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Height_Augmented_Pre.h5")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Height prediction without data augmentation-normal images


In [ ]:
best_parameters_height=tuner_height.get_best_hyperparameters(num_trials=1)[0]
model_7 = tuner_height.hypermodel.build(best_parameters_height)
model_7.summary()
score_height=[]
fold=0
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("======"*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=images[train]
  y_train=Height[train]
  x_test=images[test]
  y_test=Height[test]
  model_7.fit(x_train,y_train,callbacks=[callback],epochs=100)
  pred=model_7.predict(x_test)
  score_height.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_height)
print(f'mean={mean(score_height)}')
print(f'standard_deviation={stdev(score_height)}')
print(f'Variance={variance(score_height)}')
plot_model(model_7, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Height.png', show_shapes=True, show_layer_names=True)
model_7.save('/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Height.h5')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Height prediction without augmentation -processed images

In [ ]:
best_parameters_height_pre=tuner_height_pre.get_best_hyperparameters(num_trials=1)[0]
model_8 = tuner_height_pre.hypermodel.build(best_parameters_height_pre)
model_8.summary()
score_height_pre=[]
fold=0
for train,test in kf.split(processed_images):
  fold+=1;
  print(fold)
  print("======"*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=processed_images[train]
  y_train=Processed_Height[train]
  x_test=processed_images[test]
  y_test=Processed_Height[test]
  model_8.fit(x_train,y_train,callbacks=[callback],epochs=100)
  pred=model_8.predict(x_test)
  score_height_pre.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_height_pre)
print(f'mean={mean(score_height_pre)}')
print(f'standard_deviation={stdev(score_height_pre)}')
print(f'Variance={variance(score_height_pre)}')
plot_model(model_8, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Height_Pre.png', show_shapes=True, show_layer_names=True)
model_8.save('/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Height_Pre.h5')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Weight Model with fine tuning finding the best model


In [ ]:
tuner_weight = kt.Hyperband(model_builder, objective='mean_absolute_percentage_error', max_epochs=10, factor=3, project_name='/content/drive/MyDrive/BMI_Face_Data/Data/Weight_Test_models')
tuner_weight.search(images,Weight, epochs=50, validation_split=0.2)

In [ ]:
tuner_weight_pre = kt.Hyperband(model_builder, objective='mean_absolute_percentage_error', max_epochs=10, factor=3, project_name='/content/drive/MyDrive/BMI_Face_Data/Data/Weight_Test_models_pre')
tuner_weight_pre.search(processed_images,Processed_Weight, epochs=50, validation_split=0.2)

Weigth prediction with data augmentation-normal images

In [ ]:
best_parameters_weight_aug=tuner_weight.get_best_hyperparameters(num_trials=1)[0]
model_9 = tuner_weight.hypermodel.build(best_parameters_weight_aug)
model_9.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.1,rotation_range=20)
score_weight_aug=[]
fold=0
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("======"*10)
  x_train=images[train]
  y_train=Weight[train]
  x_test=images[test]
  y_test=Weight[test]
  batches=dataGenerator.flow(x_train,y_train,batch_size=20)
  model_9.fit_generator(batches,epochs=100,callbacks=[callback])
  pred=model_9.predict(x_test)
  score_weight_aug.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_weight_aug)
print(f'mean={mean(score_weight_aug)}')
print(f'standard_deviation={stdev(score_weight_aug)}')
print(f'Variance={variance(score_weight_aug)}')
plot_model(model_9, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Weight_Aug.png', show_shapes=True, show_layer_names=True)
model_9.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Weight_Augmented.h5")

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Weight prediction with data augmentation-processed images

In [ ]:
best_parameters_weight_aug_pre=tuner_weight_pre.get_best_hyperparameters(num_trials=1)[0]
model_10 = tuner_weight_pre.hypermodel.build(best_parameters_weight_aug_pre)
model_10.summary()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
from keras.preprocessing.image import ImageDataGenerator
dataGenerator=ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,shear_range=0.1,rotation_range=20)
score_weight_aug_pre=[]
fold=0
for train,test in kf.split(processed_images):
  fold+=1;
  print(fold)
  print("======"*10)
  x_train=processed_images[train]
  y_train=Processed_Weight[train]
  x_test=processed_images[test]
  y_test=Processed_Weight[test]
  batches=dataGenerator.flow(x_train,y_train,batch_size=20)
  model_10.fit_generator(batches,epochs=100,callbacks=[callback])
  pred=model_10.predict(x_test)
  score_weight_aug_pre.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_weight_aug_pre)
print(f'mean={mean(score_weight_aug_pre)}')
print(f'standard_deviation={stdev(score_weight_aug_pre)}')
print(f'Variance={variance(score_weight_aug_pre)}')
plot_model(model_10, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Weight_Aug_Pre.png', show_shapes=True, show_layer_names=True)
model_10.save("/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Weight_Augmented_Pre.h5")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Weight prediction without data augmentation -normal images

In [ ]:
best_parameters_weight=tuner_weight.get_best_hyperparameters(num_trials=1)[0]
model_11 = tuner_weight.hypermodel.build(best_parameters_weight)
model_11.summary()
score_weight=[]
fold=0
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("======"*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=images[train]
  y_train=Weight[train]
  x_test=images[test]
  y_test=Weight[test]
  model_11.fit(x_train,y_train,callbacks=[callback],epochs=100)
  pred=model_11.predict(x_test)
  score_weight.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_weight)
print(f'mean={mean(score_weight)}')
print(f'standard_deviation={stdev(score_weight)}')
print(f'Variance={variance(score_weight)}')
plot_model(model_11, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Weight.png', show_shapes=True, show_layer_names=True)
model_11.save('/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Weight.h5')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

Weight prediction without data augmentation -processed images

In [ ]:
best_parameters_weight_pre=tuner_weight_pre.get_best_hyperparameters(num_trials=1)[0]
model_12 = tuner_weight_pre.hypermodel.build(best_parameters_weight_pre)
model_12.summary()
score_weight_pre=[]
fold=0
for train,test in kf.split(processed_images):
  fold+=1;
  print(fold)
  print("======"*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=processed_images[train]
  y_train=Processed_Weight[train]
  x_test=processed_images[test]
  y_test=Processed_Weight[test]
  model_12.fit(x_train,y_train,callbacks=[callback],epochs=100)
  pred=model_12.predict(x_test)
  score_weight_pre.append(metrics.mean_absolute_percentage_error(pred,y_test))
  print("====="*10)
print(score_weight_pre)
print(f'mean={mean(score_weight_pre)}')
print(f'standard_deviation={stdev(score_weight_pre)}')
print(f'Variance={variance(score_weight_pre)}')
plot_model(model_12, to_file='/content/drive/MyDrive/BMI_Face_Data/Model_Graph/Weight_Pre.png', show_shapes=True, show_layer_names=True)
model_12.save('/content/drive/MyDrive/BMI_Face_Data/Prediction_Models/Weight_Pre.h5')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                      

BMI calculation from predicted height and Weight

Classification Model for BMI data Preprocessing

In [ ]:
over=SMOTE()
under=RandomUnderSampler()
images=images.reshape(1230,224*224*3)
images,BMI_classification=over.fit_resample(images,BMI_classification)
images=images.reshape(images.shape[0],224,224,3)
np.unique(BMI_classification,return_counts=True)[1]


array([474, 474, 474, 474, 474, 474])

In [ ]:
from tensorflow.keras.utils import to_categorical
BMI_classification=to_categorical(BMI_classification)

In [ ]:
def BMI_category(input):
  if input==0: return "Underweight"
  elif input==1: return " Normal Weight"
  elif input==2: return " Overweight"
  elif input==3: return " Obesity Class 1"
  elif input==4: return "Obesity Class 2"
  else: return "Obesity Class 3"

In [ ]:
model_gender.summary()

Classification Model for BMI


In [ ]:
model_9=Sequential(model_gender.layers[:-3])
for layer in model_9.layers:
  layer.trainable = False
model_9.add(Conv2D(6,(1,1),activation='relu'))
model_9.add(Flatten())
model_9.add(Dense(216,activation='relu'))
model_9.add(Dense(36,activation='relu'))
model_9.add(Dense(6,activation='softmax'))
model_9.summary()

In [ ]:
model_9.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer=keras.optimizers.Adam(learning_rate=0.001))
from statistics import mean
score_BMI_classification=[]
fold=0;
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("=========="*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
  x_train=images[train]
  y_train=BMI_classification[train]
  # x_train=x_train.reshape(x_train.shape[0],224*224*3)
  # x_train,y_train=over.fit_resample(x_train,y_train)
  # x_train=x_train.reshape(x_train.shape[0],224,224,3)
  x_test=images[test]
  y_test=BMI_classification[test]
  model_9.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[callback],epochs=100)
  pred=model_9.predict(x_test)
  score_BMI_classification.append(metrics.categorical_crossentropy(pred,y_test))
  print("====="*10)
print(score_BMI_classification)
print(mean(score_BMI_classification))
model_5.save('/content/drive/MyDrive/PROJECTS/BMI/Height_Estimation_Without_Augmentation_initial.h5')

In [ ]:
def model_builder_2(hp):
  model_=Sequential(model_gender.layers[:-3])
  for layer in model_.layers:
    layer.trainable = False
  model_.add(Conv2D(4096,(1,1),activation='relu'))
  model_.add(Flatten())
  for i in range(hp.Int('num_layers', 2, 5)): 
    model_.add(keras.layers.Dense(units=hp.Int('units_' + str(i),min_value=256,max_value=4096,step=32), activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid','softmax'], default='relu')))
  model_.add(keras.layers.Dense(6,activation='softmax'))
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  optimizer=hp.Choice('optimizer', values=['adam', 'adagrad', 'SGD', 'RMSprop'])
  if optimizer == 'adam':
    optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate)
  elif optimizer == 'adagrad':
    optimizer=keras.optimizers.Adagrad(learning_rate=hp_learning_rate)
  elif optimizer == 'SGD':
    optimizer=keras.optimizers.SGD(learning_rate=hp_learning_rate)
  elif optimizer == 'RMSprop':
    optimizer=keras.optimizers.RMSprop(learning_rate=hp_learning_rate)
  model_.compile(
      loss="categorical_crossentropy",metrics=["accuracy"],
      optimizer=optimizer
      )
  return model_

In [ ]:
tuner_BMI_classification = kt.Hyperband(model_builder_2, objective='accuracy', max_epochs=10, factor=3, directory='/content/drive/MyDrive/BMI_Face_Data/BMI_Classification', project_name='/content/drive/MyDrive/BMI_Face_Data/BMI_Classification/Best_Model')
tuner_BMI_classification.search(images,BMI_classification, epochs=50, validation_split=0.2)
best_parameters_BMI_classification=tuner_BMI_classification.get_best_hyperparameters(num_trials=1)[0]
model_8 = tuner_BMI_classification.hypermodel.build(best_parameters_BMI_classification)
model_8.summary()

In [ ]:
from statistics import mean
score_BMI_classification=[]
fold=0;
for train,test in kf.split(images):
  fold+=1;
  print(fold)
  print("=========="*10)
  callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
  x_train=images[train]
  y_train=BMI_classification[train]
  x_test=images[test]
  y_test=BMI_classification[test]
  model_8.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[callback],epochs=100)
  pred=model_8.predict(x_test)
  score_BMI_Classification.append(tf.keras.losses.categorical_crossentropy(pred,y_test))
  print("====="*10)
print(score_BMI_classification)
print(mean(score_BMI_classification))
model_5.save('/content/drive/MyDrive/PROJECTS/BMI/Height_Estimation_Without_Augmentation.h5')

In [ ]:
model_5=tf.keras.models.load_model('/content/drive/MyDrive/BMI_Face_Data/Weight_Estimation_Augmented_img_pre.h5')
img=cv2.imread('/content/drive/MyDrive/BMI_Face_Data/IMG2.jpg')
img=cv2.resize(img,(224,224))
img=img.reshape(1,224,224,3)
model_5.predict(img)

array([[60.8003]], dtype=float32)